# Natural Language Processing - lab 6 (Language modelling)

Bartosz Klimza

# Necessary imports

In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 25.5 MB/s 
     |████████████████████████████████| 895 kB 42.9 MB/s 
     |████████████████████████████████| 61 kB 503 kB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 596 kB 43.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [24]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# 1. Read the documentation of Language modelling in the Transformers library.

In [29]:
# https://huggingface.co/docs/transformers/task_summary#language-modeling

# 2. Download three Polish models from the Huggingface repository:

In [25]:
tokenizer_1 = AutoTokenizer.from_pretrained("dkleczek/bert-base-polish-uncased-v1")
model_1 = AutoModelForMaskedLM.from_pretrained("dkleczek/bert-base-polish-uncased-v1")

tokenizer_2 = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-pl-cased")
model_2 = AutoModelForMaskedLM.from_pretrained("Geotrend/distilbert-base-pl-cased")

tokenizer_3 = AutoTokenizer.from_pretrained("Geotrend/bert-base-pl-cased")
model_3 = AutoModelForMaskedLM.from_pretrained("Geotrend/bert-base-pl-cased")

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# 3. Produce the predictions for the following sentences (use each model and check 5 predictions):

* (M) Warszawa to największe [MASK].
* (D) Te zabawki należą do [MASK].
* (C) Policjant przygląda się [MASK].
* (B) Na środku skrzyżowania widać [MASK].
* (N) Właściciel samochodu widział złodzieja z [MASK].
* (Ms) Prezydent z premierem rozmawiali wczoraj o [MASK].
* (W) Witaj drogi [MASK].

In [30]:
# This method was described in a documentation from the first point
def make_prediction(tokenizer, model, sequence):
  sequence = sequence.replace("[MASK]", tokenizer.mask_token)
  inputs = tokenizer(sequence, return_tensors="pt")
  mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
  token_logits = model(**inputs).logits
  mask_token_logits = token_logits[0, mask_token_index, :]
  top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
  return_list = []
  for token in top_5_tokens:
    return_list.append(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))
  return return_list

In [27]:
ex_3_sequences = ["Warszawa to największe [MASK].",
                  "Te zabawki należą do [MASK].",
                  "Policjant przygląda się [MASK].",
                  "Na środku skrzyżowania widać [MASK].",
                  "Właściciel samochodu widział złodzieja z [MASK].",
                  "Prezydent z premierem rozmawiali wczoraj o [MASK].",
                  "Witaj, drogi [MASK]."]

In [28]:
print("dkleczek/bert-base-polish-uncased-v1")
for s in ex_3_sequences:
  print(make_prediction(tokenizer_1, model_1, s))

print("\n")
print("Geotrend/distilbert-base-pl-cased")

for s in ex_3_sequences:
  print(make_prediction(tokenizer_2, model_2, s))

print("\n")
print("Geotrend/bert-base-pl-cased")

for s in ex_3_sequences:
  print(make_prediction(tokenizer_3, model_3, s))

dkleczek/bert-base-polish-uncased-v1
['Warszawa to największe miasto.', 'Warszawa to największe miejsce.', 'Warszawa to największe lotnisko.', 'Warszawa to największe centrum.', 'Warszawa to największe miasta.']
['Te zabawki należą do mnie.', 'Te zabawki należą do ciebie.', 'Te zabawki należą do niego.', 'Te zabawki należą do nas.', 'Te zabawki należą do pana.']
['Policjant przygląda się temu.', 'Policjant przygląda się mu.', 'Policjant przygląda się im.', 'Policjant przygląda się nam.', 'Policjant przygląda się jej.']
['Na środku skrzyżowania widać dym.', 'Na środku skrzyżowania widać wszystko.', 'Na środku skrzyżowania widać ulice.', 'Na środku skrzyżowania widać ruch.', 'Na środku skrzyżowania widać samochod.']
['Właściciel samochodu widział złodzieja z tyłu.', 'Właściciel samochodu widział złodzieja z bliska.', 'Właściciel samochodu widział złodzieja z samochodu.', 'Właściciel samochodu widział złodzieja z kamera.', 'Właściciel samochodu widział złodzieja z parkingu.']
['Prezydent 

# 4. Check the model predictions for the following sentences (using each model):

* Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].
* Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].

In [31]:
ex_4_sequences = ["Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].",
                  "Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK]."]

In [32]:
print("dkleczek/bert-base-polish-uncased-v1")
for s in ex_4_sequences:
  print(make_prediction(tokenizer_1, model_1, s))

print("\n")
print("Geotrend/distilbert-base-pl-cased")

for s in ex_4_sequences:
  print(make_prediction(tokenizer_2, model_2, s))

print("\n")
print("Geotrend/bert-base-pl-cased")

for s in ex_4_sequences:
  print(make_prediction(tokenizer_3, model_3, s))

dkleczek/bert-base-polish-uncased-v1
['Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.', 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie martwił.', 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgodził.', 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie przyznał.', 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie przejmował.']
['Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie dowiedziała.', 'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zgodziła.', 'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie przyznała.', 'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zabiła.', 'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie poddała.']


Geotrend/distilbert-base-pl-cased
['Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie było.', 'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się 

# 5. Check the model predictions for the following sentences:

* [MASK] wrze w temperaturze 100 stopni, a zamarza w temperaturze 0 stopni Celsjusza.
* W wakacje odwiedziłem [MASK], który jest stolicą Islandii.
* Informatyka na [MASK] należy do najlepszych kierunków w Polsce.

In [33]:
ex_5_sequences = [
  "[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.",
  "W wakacje odwiedziłem [MASK], który jest stolicą Islandii.",
  "Informatyka na [MASK] należy do najlepszych kierunków w Polsce.",
]

In [34]:
print("dkleczek/bert-base-polish-uncased-v1")
for s in ex_5_sequences:
  print(make_prediction(tokenizer_1, model_1, s))

print("\n")
print("Geotrend/distilbert-base-pl-cased")

for s in ex_5_sequences:
  print(make_prediction(tokenizer_2, model_2, s))

print("\n")
print("Geotrend/bert-base-pl-cased")

for s in ex_5_sequences:
  print(make_prediction(tokenizer_3, model_3, s))

dkleczek/bert-base-polish-uncased-v1
['woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.', 'ciało wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.', 'nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.', 'lod wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.', 'powietrze wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.']
['W wakacje odwiedziłem zamek, który jest stolicą Islandii.', 'W wakacje odwiedziłem kraj, który jest stolicą Islandii.', 'W wakacje odwiedziłem sztokholm, który jest stolicą Islandii.', 'W wakacje odwiedziłem park, który jest stolicą Islandii.', 'W wakacje odwiedziłem region, który jest stolicą Islandii.']
['Informatyka na uniwersytecie należy do najlepszych kierunków w Polsce.', 'Informatyka na swiecie należy do najlepszych kierunków w Polsce.', 'Informatyka na uczelni należy do najlepszych kierunków w Polsce.', 'Informatyka na uc

# 6. If you want to use causal language models such as PapuGaPT2 or plT5, you should change the last three examples to accomodate for the fact, that these models are better suited for causal language modelling.

I  used neither PapuGaPT2 nor plT5

# 7. Answer the following questions:

* Which of the models produced the best results?

Moim zdaniem najlepiej poradził sobie model dkleczek/bert-base-polish-uncased-v1.

* Was any of the models able to capture Polish grammar?

Model_1 i model_2 poradziły sobie całkiem dobrze, jednak mimo wszystko pojawiały się błędy. Model_3 wypadł najgorzej, co widać szczególnie w przykładzie 7 w zadaniu 3.

* Was any of the models able to capture long-distant relationships between the words?

Można uznać, że model dkleczek/bert-base-polish-uncased-v1 jest w stanie dostrzegać takie zależności.

* Was any of the models able to capture world knowledge?

Ciężko powiedzieć, Geotrend/bert-base-pl-cased odgadł stolicę Islandii, ale dkleczek/bert-base-polish-uncased-v1 wiedział co wrze w 100 stopniach i zamarza w 0.

* What are the most striking errors made by the models?

  * Witaj, drogi ##arz. (Zadanie 3, model_3)
  * W wakacje odwiedziłem sztokholm, który jest stolicą Islandii. (Zadanie 5, model_1)
  * Informatyka na ulicy należy do najlepszych kierunków w Polsce. (Zadanie 5, model_2)